In [1]:
import requests 
import json 
import urllib.request
import datetime
import time
from tqdm import tqdm 
import pandas as pd
import folium 
import webbrowser
from folium.plugins import MarkerCluster, MiniMap

In [8]:
def getMaskStoreInfo():
    url = "https://8oi9s0nnth.apigw.ntruss.com/corona19-masks/v1/storesByAddr/json" 
    req = requests.get(url) 
    address = req.json()['address'] 
    counts = req.json()['count'] 
    re_stat_num=0
    stock_num=0
    addrs = [] 
    codes = [] 
    created_ats=[]
    latitudes = [] 
    longitudes = [] 
    names = []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
    remain_stats=[]
    stock_ats=[]
    types = []     
    req = requests.get(url) 
    store_infos = req.json()['stores'] 

    
    for i in range(len(store_infos)):  
        addrs.append(store_infos[i]['addr']) 
        codes.append(store_infos[i]['code']) 
        created_ats.append(store_infos[i]['created_at'])    
        latitudes.append(store_infos[i]['lat']) 
        longitudes.append(store_infos[i]['lng']) 
        names.append(store_infos[i]['name'])  
         
        try:
            store_infos[i]['remain_stat']
        except:
            remain_stats.append('null')   
        else:
            if store_infos[i]['remain_stat']:
                remain_stats.append(store_infos[i]['remain_stat']) 
            else :
                remain_stats.append('null') 
                
        try:
            store_infos[i]['stock_at']
        except:
            stock_ats.append('null')   
        else:
            if store_infos[i]['stock_at']:
                stock_ats.append(store_infos[i]['stock_at']) 
            else :
                stock_ats.append('null')         
                
        types.append(store_infos[i]['type'])  
        
    print(len(remain_stats))
    mask_store_info_df = pd.DataFrame({"addr":addrs, "code":codes, "created_at":created_ats,"latitude":latitudes, "longitude":longitudes, "name":names, "remain_stat":remain_stats,"stock_at":stock_ats,"type":types}) 
    return mask_store_info_df

        
def main():     
    color_dic = {'plenty':'green', 'some':'orange', 'few':'red', 'empty':'gray', 'break':'black','null':'blue'}
    remain_stat_kor = {'plenty':'100개 이상', 'some':'30개 이상', "few":'2개-30개', "empty":"재고 없음", "break":"문닫음",'null':'정보없음'}
    
    my_info_df = getMaskStoreInfo()
    my_info_df.to_csv("mask_store_info.csv")
    data_for_draw = my_info_df.loc[:, ['name', 'latitude', 'longitude', 'remain_stat','addr']] 
    data_for_draw_nan = data_for_draw.dropna() 
    
    
    map_hs = folium.Map((37.5028561, 127.0447712), zoom_start = 14) 
    mc = MarkerCluster() 
    names = list(data_for_draw_nan['name']) 
    latitudes = list(data_for_draw_nan['latitude']) 
    longitudes = list(data_for_draw_nan['longitude']) 
    remain_stats = list(data_for_draw_nan['remain_stat'])
    addrs = list(data_for_draw_nan['addr'])
    
    for i in tqdm(range(len(names))): 
        popups = folium.Popup ('<'+names[i]+'>'+' '+addrs[i] + ' ' + remain_stat_kor[remain_stats[i]],max_width = 350, min_width = 200)
        mc.add_child(folium.Marker(location = [latitudes[i], longitudes[i]], icon=folium.Icon(color=color_dic[remain_stats[i]]), popup=popups)) 
        map_hs.add_child(mc)

    map_hs.save('corona-mask.html')

In [9]:
if __name__ == "__main__":   
    main()

100%|█████████████████████████████████████████████████████████████████████████████| 463/463 [00:00<00:00, 10510.50it/s]

463
